In [1]:
import argparse
import json
import os
import pandas as pd
import os

import datetime
import numpy as np
from cgms_data_seg_diatrend import CGMSDataSeg
from cnn_ohio import regressor, regressor_transfer, test_ckpt
from data_reader_DiaTrend import DataReader

In [2]:
import tensorflow as tf

# New method in TensorFlow 2.x:
# This will list the devices TensorFlow recognizes
print("TensorFlow version:", tf.__version__)
print("List of devices available to TensorFlow:")
print(tf.config.list_physical_devices())



TensorFlow version: 2.10.1
List of devices available to TensorFlow:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
# Example subjects
file_names = [
    'Subject11',
    'Subject26', 
    'Subject3', 
    'Subject30', 
    'Subject31', 
    'Subject36', 
    'Subject15', 
    'Subject37', 
    'Subject38', 
    'Subject39', 
    'Subject41', 
    'Subject42', 
    'Subject43',
    'Subject5', 
    'Subject6', 
    'Subject8', 
]

In [ ]:
for subj in file_names:
    subject = pd.read_excel(f"C:/Users/username/OneDrive/Desktop/BGprediction/DiaTrend/dataset/{subj}.xlsx","CGM")
    split_index = int(len(subject) * 0.8)
    # Split the DataFrame
    train_df = subject[:split_index]
    test_df = subject[split_index:]

    # Save the DataFrames to CSV files
    train_df.to_csv(f'C:/Users/username/OneDrive/Desktop/BGprediction/DiaTrend/train/{subj}_training_data.csv', index=False)
    test_df.to_csv(f'C:/Users/username/OneDrive/Desktop/BGprediction/DiaTrend/test/{subj}_testing_data.csv', index=False)

    # Optionally, confirm the operation
    print(f"Training data saved with {len(train_df)} records.")
    print(f"Testing data saved with {len(test_df)} records.")

In [ ]:
# Need to do the train_test split
# First 80% in training
# Last 20% in test
# split_index = int(len(subject) * 0.8)
# # Split the DataFrame
# train_df = subject[:split_index]
# test_df = subject[split_index:]

# # Save the DataFrames to CSV files
# train_df.to_csv(f'C:/Users/username/OneDrive/Desktop/BGprediction/DiaTrend/train/{subj}_training_data.csv', index=False)
# test_df.to_csv(f'C:/Users/username/OneDrive/Desktop/BGprediction/DiaTrend/test/{subj}_testing_data.csv', index=False)

# # Optionally, confirm the operation
# print(f"Training data saved with {len(train_df)} records.")
# print(f"Testing data saved with {len(test_df)} records.")



In [4]:
# Assuming your DataFrame is named df
subject['date'] = pd.to_datetime(subject['date'])  # Convert 'date' column to datetime if not already
subject.sort_values('date', inplace=True)  # Sort the DataFrame by the 'date' column


In [11]:
# Assuming self.interval_timedelta is set, for example:
interval_timedelta = datetime.timedelta(minutes=6)  # Example timedelta of 6 minutes, providing a range for latency

# Create a list to store the results
res = []

# Initialize the first group
if not subject.empty:
    current_group = [subject.iloc[0]['mg/dl']]
    last_time = subject.iloc[0]['date']

# Iterate over rows in DataFrame starting from the second row
for index, row in subject.iloc[1:].iterrows():
    current_time = row['date']
    if (current_time - last_time) <= interval_timedelta:
        # If the time difference is within the limit, add to the current group
        current_group.append(row['mg/dl'])
    else:
        # Otherwise, start a new group
        res.append(current_group)
        current_group = [row['mg/dl']]
    last_time = current_time

# Add the last group if it's not empty
if current_group:
    res.append(current_group)


# Functions

In [4]:
def preprocess_DiaTrend(path):

    subject = pd.read_csv(path)
    subject['date'] = pd.to_datetime(subject['date'], errors='coerce')  # Convert 'date' column to datetime if not already
    print(subject['date'][0])
    subject.sort_values('date', inplace=True)  # Sort the DataFrame by the 'date' column

    # Assuming self.interval_timedelta is set, for example:
    interval_timedelta = datetime.timedelta(minutes=6)  # Example timedelta of 6 minutes, providing a range for latency

    # Create a list to store the results
    res = []

    # Initialize the first group
    if not subject.empty:
        current_group = [subject.iloc[0]['mg/dl']]
        last_time = subject.iloc[0]['date']

    # Iterate over rows in DataFrame starting from the second row
    for index, row in subject.iloc[1:].iterrows():
        current_time = row['date']
        if (current_time - last_time) <= interval_timedelta:
            # If the time difference is within the limit, add to the current group
            current_group.append(row['mg/dl'])
        else:
            # Otherwise, start a new group
            res.append(current_group)
            current_group = [row['mg/dl']]
        last_time = current_time

    # Add the last group if it's not empty
    if current_group:
        res.append(current_group)
    
    # Filter out groups with fewer than 10 glucose readings
    # res = [group for group in res if len(group) >= 10]

    return res

# For loop to generate res for train and test

In [ ]:
# Fomulate a loop to create a list to include all the files in train and test datset and generate the res for each of them seperately

# Define the directory path
train_directory_path = r'C:\Users\username\OneDrive\Desktop\BGprediction\DiaTrend\train'  # Use a raw string for paths on Windows

# List files without their extensions
train_file_names = [os.path.splitext(file)[0] for file in os.listdir(train_directory_path)
              if os.path.isfile(os.path.join(train_directory_path, file))]

# Print the list of file names
print(train_file_names)


In [ ]:
cleaned_subjects = [s.replace("_training_data", "") for s in train_file_names]

print(cleaned_subjects)

In [ ]:
# Define the directory path
test_directory_path = r'C:\Users\username\OneDrive\Desktop\BGprediction\DiaTrend\test'  # Use a raw string for paths on Windows

# List files without their extensions
test_file_names = [os.path.splitext(file)[0] for file in os.listdir(test_directory_path)
              if os.path.isfile(os.path.join(test_directory_path, file))]

# Print the list of file names
print(test_file_names)

In [ ]:
train_data = dict()
for subj in train_file_names:
    print(subj)
    subj_path = f'C:/Users/username/OneDrive/Desktop/BGprediction/DiaTrend/train/{subj}.csv'
    reader = preprocess_DiaTrend(subj_path)
    train_data[subj] = reader

In [ ]:
# Have not been run
test_data = dict()
for subj in test_file_names:
    print(subj)
    subj_path = f'C:/Users/username/OneDrive/Desktop/BGprediction/DiaTrend/test/{subj}.csv'
    reader = preprocess_DiaTrend(subj_path)
    test_data[subj] = reader

In [10]:
epoch = 10
ph = 6
path = "../diatrend_results"

In [11]:
# a dumb dataset instance
train_dataset = CGMSDataSeg(
    "ohio", "C:/Users/username/OneDrive/Desktop/BGprediction/DiaTrend/train/Subject11_training_data.csv", 5
)
sampling_horizon = 12
prediction_horizon = ph
scale = 0.01
outtype = "Same"
# train on training dataset
# k_size, nblock, nn_size, nn_layer, learning_rate, batch_size, epoch, beta
with open(f'../diatrend_results/config.json') as json_file:
    config = json.load(json_file)
argv = (
    config["k_size"],
    config["nblock"],
    config["nn_size"],
    config["nn_layer"],
    config["learning_rate"],
    config["batch_size"],
    epoch,
    config["beta"],
)
l_type = config["loss"]
# test on patients data
outdir = os.path.join(path, f"ph_{prediction_horizon}_{l_type}")
if not os.path.exists(outdir):
    os.makedirs(outdir)
all_errs = []

2018-06-19 16:19:32
Reading 571 segments


In [ ]:
train_data.keys()

In [ ]:
train_pids = set(cleaned_subjects) - set([pid])
train_pids

In [ ]:
# Loop
cleaned_subjects.sort()
standard = False  # do not use standard
all_errs = []
for pid in cleaned_subjects: # First 9 as subset, can be an example
    train_pids = set(cleaned_subjects) - set([pid])
    local_train_data = []
    for k in train_pids:
        local_train_data += train_data[k + "_training_data"]
    print(f"Pretrain data: {sum([sum(x) for x in local_train_data])}")
    
    train_dataset.data = local_train_data
    train_dataset.set_cutpoint = -1
    train_dataset.reset(
        sampling_horizon,
        prediction_horizon,
        scale,
        100,
        False,
        outtype,
        1,
        standard,
    )
    regressor(train_dataset, *argv, l_type, outdir)
    # Fine-tune and test
    target_test_dataset = CGMSDataSeg(
    "ohio", f"C:/Users/username/OneDrive/Desktop/BGprediction/DiaTrend/test/{pid}_testing_data.csv", 5
    )
    target_test_dataset.set_cutpoint = 1
    target_test_dataset.reset(
        sampling_horizon,
        prediction_horizon,
        scale,
        0.01,
        False,
        outtype,
        1,
        standard,
    )

    target_train_dataset = CGMSDataSeg(
    "ohio", f"C:/Users/username/OneDrive/Desktop/BGprediction/DiaTrend/train/{pid}_training_data.csv", 5
    )
    target_train_dataset.set_cutpoint = -1
    target_train_dataset.reset(
        sampling_horizon,
        prediction_horizon,
        scale,
        100,
        False,
        outtype,
        1,
        standard,
    )
    err, labels = test_ckpt(target_test_dataset, outdir)
    errs = [err]
    transfer_res = [labels]
    for i in range(1, 2):
        err, labels = regressor_transfer(
            target_train_dataset,
            target_test_dataset,
            config["batch_size"],
            epoch,
            outdir,
            i,
        )
        errs.append(err)
        transfer_res.append(labels)
    transfer_res = np.concatenate(transfer_res, axis=1)
    np.savetxt(
        f"{outdir}/{pid}.txt",
        transfer_res,
        fmt="%.4f %.4f %.4f %.4f",
    )
    all_errs.append([pid] + errs)
all_errs = np.array(all_errs)
np.savetxt(f"{outdir}/errors.txt", all_errs, fmt="%s %.4f %.4f")
# label pair:(groundTruth, y_pred)

In [ ]:
all_errs

In [ ]:
# Convert the second and third columns to floats
second_column = all_errs[:, 1].astype(float)
third_column = all_errs[:, 2].astype(float)

# Calculate the average
average_second_column = np.mean(second_column)
average_third_column = np.mean(third_column)

print("Average of the second column:", average_second_column)
print("Average of the third column:", average_third_column)

In [38]:
# Convert the second and third columns to floats
second_column = all_errs[:, 1].astype(float)
third_column = all_errs[:, 2].astype(float)

# Calculate the average
average_second_column = np.mean(second_column)
average_third_column = np.mean(third_column)

print("Average of the second column:", average_second_column)
print("Average of the third column:", average_third_column)

Average of the second column: 0.23931833875
Average of the third column: 0.23320475625


In [ ]:
# Also, if the previous cell run into an issue but all result txt files are ready
# You can run this to evaluate:
import argparse
import json
import os
import pandas as pd
import os
import datetime
import numpy as np

# List all files and directories in the current directory
files_and_directories = os.listdir('.')

# Filter for files that end with .txt
txt_files = [file for file in files_and_directories if file.endswith('.txt')]


# Read the data from the text file
def calcuate_rmse(file):
    data = np.loadtxt(file)  # Make sure to replace 'data.txt' with your actual file path
    print(file)
    # Splitting the data into groundtruth and predictions
    groundtruth = data[:, 0]  # First column as ground truth (also same as third column)
    predictions_1 = data[:, 1]  # Second column as predictions from method 1
    predictions_2 = data[:, 3]  # Fourth column as predictions from method 2

    # Function to calculate RMSE
    def calculate_rmse(true_values, predictions):
        mse = np.mean((true_values - predictions) ** 2)
        rmse = np.sqrt(mse)
        return rmse

    # Calculate RMSE for each method
    rmse_method_1 = calculate_rmse(groundtruth, predictions_1)
    rmse_method_2 = calculate_rmse(groundtruth, predictions_2)

    print("RMSE for Method 1:", rmse_method_1)
    print("RMSE for Method 2:", rmse_method_2)
    return rmse_method_1


rmse_list = []
for f in txt_files[1:]:
    rmse1 = calcuate_rmse(f)
    print(rmse1)
    rmse_list.append(rmse1)

print(np.average(rmse_list))

print(rmse_list)